In [1]:
import glob
import pandas as pd

from collections import defaultdict
import os


In [2]:
# sample_dict = {}
sample_dict = {sample.split('/')[5]: [] for sample in glob.glob('/private/nanopore/UCSC_HPRC_nanopore_Year4/basecalled/*/*')}
for sample in glob.glob('/private/nanopore/UCSC_HPRC_nanopore_Year4/basecalled/*/*/*'):
	sample_dict[sample.split('/')[5]].append(sample) 
     


In [96]:

# New dictionary with updated structure
new_dict = defaultdict(list)

# Populate the new dictionary
for key, file_paths in sample_dict.items():
    for file_path in file_paths:
        filename = os.path.basename(file_path)
        # if not filename.endswith('summary_stats.txt'):
        if filename.endswith('summary_stats.txt'):
            file_key = f"{filename.split('_')[0]}_{filename.split('_')[1]}_{filename.split('_')[2]}_{filename.split('_')[3]}_{filename.split('_')[4]}_{filename.split('_')[5]}_{filename.split('_')[6]}"
            new_dict[file_key].append(file_path)

# Convert defaultdict to a regular dictionary
new_dict = dict(new_dict)

# Prepare data for DataFrame
data = []
for file_id, files in new_dict.items():
    ont_reads = next((f for f in files if f.endswith('.bam')), '')
    ont_sequencing_summary_files = next((f for f in files if f.endswith('.txt')), '')
    data.append({
        'file_id': file_id,
        'ont_reads': ont_reads,
        'ont_sequencing_summary_files': ont_sequencing_summary_files
    })

# Create DataFrame
df = pd.DataFrame(data)
df['sample_aggregate']= [file.split('_')[-2] for file in df['file_id'].tolist()]

In [97]:
df

,file_id,ont_reads,ont_sequencing_summary_files,sample_aggregate
0,09_06_23_R1041_HPRC_GM19131_3,,/private/nanopore/UCSC_HPRC_nanopore_Year4/bas...,GM19131
1,09_06_23_R1041_HPRC_GM19131_2,,/private/nanopore/UCSC_HPRC_nanopore_Year4/bas...,GM19131
2,09_06_23_R1041_HPRC_GM19131_1,,/private/nanopore/UCSC_HPRC_nanopore_Year4/bas...,GM19131
3,10_10_23_R1041_HPRC_HG03103_3,,/private/nanopore/UCSC_HPRC_nanopore_Year4/bas...,HG03103
4,10_10_23_R1041_HPRC_HG03103_2,,/private/nanopore/UCSC_HPRC_nanopore_Year4/bas...,HG03103
...,...,...,...,...
383,12_05_23_R1041_HPRC_GM21144_2,,/private/nanopore/UCSC_HPRC_nanopore_Year4/bas...,GM21144
384,12_05_23_R1041_HPRC_GM21144_3,,/private/nanopore/UCSC_HPRC_nanopore_Year4/bas...,GM21144
385,12_05_23_R1041_HPRC_GM21144_1,,/private/nanopore/UCSC_HPRC_nanopore_Year4/bas...,GM21144
386,01_23_24_R1041_HPRC_GM21144_2,,/private/nanopore/UCSC_HPRC_nanopore_Year4/bas...,GM21144


In [98]:
ont_batch1 = pd.read_csv('ont_samples_output_table.csv')
ont_batch2 = pd.read_csv('ont_samples_batch2_output_table.csv')
ont_df = pd.concat([ont_batch1, ont_batch2])

In [99]:
assert ont_df.shape[0] == df.shape[0]

In [100]:
ntsm_list = []
for ntsm_data in ont_df['ntsm_workflow.ntsm_eval_out'].tolist():
	ntsm_list.append(float(pd.read_csv(ntsm_data,sep='\t',header=None)[2][0]))

ont_df['ntsm_score'] = ntsm_list

In [101]:
ont_df

,sample_id,input_reads_1,input_reads_2,read_1_type,read_2_type,ntsm_workflow.ntsm_eval_out,ntsm_workflow.ntsm_count_2,ntsm_workflow.ntsm_count_1,ntsm_score
0,10_10_23_R1041_HPRC_HG03103_3,['/private/nanopore/UCSC_HPRC_nanopore_Year4/b...,['/private/groups/hprc/human-pangenomics/1000G...,ont,other,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,0.409534
1,10_10_23_R1041_HPRC_HG03103_2,['/private/nanopore/UCSC_HPRC_nanopore_Year4/b...,['/private/groups/hprc/human-pangenomics/1000G...,ont,other,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,0.406655
2,10_10_23_R1041_HPRC_HG03103_1,['/private/nanopore/UCSC_HPRC_nanopore_Year4/b...,['/private/groups/hprc/human-pangenomics/1000G...,ont,other,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,0.456458
3,01_16_24_R1041_HPRC_HG00321_1,['/private/nanopore/UCSC_HPRC_nanopore_Year4/b...,['/private/groups/hprc/human-pangenomics/1000G...,ont,other,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,0.353827
4,11_7_23_R1041_HPRC_HG00321_2,['/private/nanopore/UCSC_HPRC_nanopore_Year4/b...,['/private/groups/hprc/human-pangenomics/1000G...,ont,other,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,0.806872
...,...,...,...,...,...,...,...,...,...
174,12_05_23_R1041_HPRC_GM21144_1,/private/nanopore/UCSC_HPRC_nanopore_Year4/bas...,['/private/groups/hprc/human-pangenomics/1000G...,ont,other,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,0.500587
175,01_23_24_R1041_HPRC_GM21144_1,/private/nanopore/UCSC_HPRC_nanopore_Year4/bas...,['/private/groups/hprc/human-pangenomics/1000G...,ont,other,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,0.437965
176,12_05_23_R1041_HPRC_GM21144_2,/private/nanopore/UCSC_HPRC_nanopore_Year4/bas...,['/private/groups/hprc/human-pangenomics/1000G...,ont,other,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,0.465377
177,01_23_24_R1041_HPRC_GM21144_2,/private/nanopore/UCSC_HPRC_nanopore_Year4/bas...,['/private/groups/hprc/human-pangenomics/1000G...,ont,other,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,0.411664


In [102]:
df = df[['file_id','ont_sequencing_summary_files']]
df.columns = ['sample_id','ont_sequencing_summary_files']
sample_file = pd.merge(df, ont_df, on='sample_id', how='left')


In [116]:
seq_summary_list = []
for seq_summary in sample_file[['sample_id' , 'ont_sequencing_summary_files']].values:
	seq_df = pd.read_csv(seq_summary[1], sep='\t')
	seq_df['sample_id'] = seq_summary[0]
	seq_summary_list.append(seq_df)
	# seq_summary_dict[seq_summary[0]] = pd.read_csv(seq_summary[1], sep='\t')
	# seq_summary_df_list[] (pd.read_csv(seq_summary, sep='\t'))
	

In [120]:
seq_summary_df = pd.concat(seq_summary_list)

In [122]:
sample_file_df = pd.merge(sample_file, seq_summary_df, on='sample_id', how='left')

In [131]:
sample_file_df['Sample']

0      GM19131_3 
1      GM19131_2 
2      GM19131_1 
3      HG03103_3 
4      HG03103_2 
          ...    
383    GM21144_2 
384    GM21144_3 
385    GM21144_1 
386    GM21144_2 
387    GM21144_1 
Name: Sample, Length: 388, dtype: object

In [132]:
sample_file_df['Sample'] = sample_file_df['Sample'].astype(str).str.strip()
sample_file_df['Sample'] = sample_file_df['Sample'].apply(lambda x: x.replace('PG', 'HG') if x.startswith('PG') else x).tolist()
sample_file_df['Sample'] = sample_file_df['Sample'].apply(lambda x: x.replace('GM', 'NA') if x.startswith('GM') else x).tolist()


In [137]:
sample_file_df['sample_ID'] = [sample.split('_')[0] for sample in sample_file_df['Sample'].tolist()]

In [139]:
biosample_df = pd.read_csv('/private/groups/hprc/human-pangenomics/HPRC_metadata/data/production/table_download_6.17.23.tsv',sep='\t')

In [141]:
set(sample_file_df[~sample_file_df['sample_ID'].isin(biosample_df['BioSample.name'])]['sample_ID'].tolist())

{'NA18523', 'NA18638', 'NA19315', 'NA21101'}

In [142]:
required_5_readstats = pd.read_csv('/private/groups/hprc/human-pangenomics/HPRC_metadata/submissions/aggregate-sample-inputs/hprc_5_readstat_sample_aggregate_rules_ONT.tsv',sep='\t', index_col=[0]).to_dict()['0']

sample_file_df.groupby('sample_ID').agg(required_5_readstats).reset_index().sort_values('100kb+').to_csv('UCSC_HPRC_ONT_Year4_sample_aggregate.csv',index=False)
# required_5_readstats_columns = required_5_readstats_columns['5_readstats_columns'].tolist()

In [146]:
sample_file_df.groupby('sample_ID').agg(required_5_readstats).reset_index().sort_values('100kb+')

,sample_ID,read_N50,coverage,100kb+,200kb+,300kb+,400kb+,500kb+,1Mb+,whales
43,HG03372,92241.666667,9.20,4.20,1.00,0.17,0.03,0.00,0.00,1
57,NA18505,54186.000000,80.43,17.18,2.45,0.37,0.08,0.02,0.00,2
40,HG03270,56221.750000,73.81,17.28,2.46,0.29,0.03,0.00,0.00,1
48,HG03583,86248.333333,43.13,18.21,4.53,0.90,0.17,0.04,0.00,1
54,HG03919,61540.714286,89.23,19.22,3.67,0.51,0.05,0.01,0.00,3
...,...,...,...,...,...,...,...,...,...,...
73,NA19131,93852.000000,82.64,38.63,11.57,3.16,0.92,0.29,0.00,28
69,NA19012,98795.666667,82.66,38.71,12.07,2.69,0.50,0.12,0.01,24
27,HG01766,88226.666667,91.74,40.32,11.36,2.60,0.56,0.14,0.00,3
58,NA18508,105497.333333,79.70,42.12,13.24,3.14,0.60,0.12,0.00,7


In [147]:
sample_file_df.to_csv('UCSC_HPRC_ONT_Year4_summary.csv',index=False)

,sample_id,ont_sequencing_summary_files,input_reads_1,input_reads_2,read_1_type,read_2_type,ntsm_workflow.ntsm_eval_out,ntsm_workflow.ntsm_count_2,ntsm_workflow.ntsm_count_1,ntsm_score,...,400kb+,500kb+,1Mb+,whales,q5,q10,q15,q20,q25,sample_ID
0,09_06_23_R1041_HPRC_GM19131_3,/private/nanopore/UCSC_HPRC_nanopore_Year4/bas...,/private/nanopore/UCSC_HPRC_nanopore_Year4/bas...,['/private/groups/hprc/human-pangenomics/1000G...,ont,other,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,0.469401,...,0.31,0.10,0.0,13,26.67,24.67,22.67,18.97,4.80,NA19131
1,09_06_23_R1041_HPRC_GM19131_2,/private/nanopore/UCSC_HPRC_nanopore_Year4/bas...,/private/nanopore/UCSC_HPRC_nanopore_Year4/bas...,['/private/groups/hprc/human-pangenomics/1000G...,ont,other,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,0.463818,...,0.31,0.10,0.0,8,30.10,27.93,25.79,21.69,6.27,NA19131
2,09_06_23_R1041_HPRC_GM19131_1,/private/nanopore/UCSC_HPRC_nanopore_Year4/bas...,/private/nanopore/UCSC_HPRC_nanopore_Year4/bas...,['/private/groups/hprc/human-pangenomics/1000G...,ont,other,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,0.440698,...,0.30,0.09,0.0,7,25.42,23.15,21.24,17.48,4.61,NA19131
3,10_10_23_R1041_HPRC_HG03103_3,/private/nanopore/UCSC_HPRC_nanopore_Year4/bas...,['/private/nanopore/UCSC_HPRC_nanopore_Year4/b...,['/private/groups/hprc/human-pangenomics/1000G...,ont,other,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,0.409534,...,0.15,0.03,0.0,1,16.87,15.39,14.05,11.38,2.42,HG03103
4,10_10_23_R1041_HPRC_HG03103_2,/private/nanopore/UCSC_HPRC_nanopore_Year4/bas...,['/private/nanopore/UCSC_HPRC_nanopore_Year4/b...,['/private/groups/hprc/human-pangenomics/1000G...,ont,other,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,0.406655,...,0.16,0.03,0.0,0,16.63,15.05,13.70,11.21,2.72,HG03103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,12_05_23_R1041_HPRC_GM21144_2,/private/nanopore/UCSC_HPRC_nanopore_Year4/bas...,/private/nanopore/UCSC_HPRC_nanopore_Year4/bas...,['/private/groups/hprc/human-pangenomics/1000G...,ont,other,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,0.465377,...,0.01,0.00,0.0,0,13.01,11.69,10.24,7.09,1.02,NA21144
384,12_05_23_R1041_HPRC_GM21144_3,/private/nanopore/UCSC_HPRC_nanopore_Year4/bas...,/private/nanopore/UCSC_HPRC_nanopore_Year4/bas...,['/private/groups/hprc/human-pangenomics/1000G...,ont,other,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,0.510650,...,0.02,0.00,0.0,0,12.31,11.36,10.52,8.66,2.62,NA21144
385,12_05_23_R1041_HPRC_GM21144_1,/private/nanopore/UCSC_HPRC_nanopore_Year4/bas...,/private/nanopore/UCSC_HPRC_nanopore_Year4/bas...,['/private/groups/hprc/human-pangenomics/1000G...,ont,other,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,0.500587,...,0.01,0.00,0.0,1,10.05,9.18,8.43,6.85,1.94,NA21144
386,01_23_24_R1041_HPRC_GM21144_2,/private/nanopore/UCSC_HPRC_nanopore_Year4/bas...,/private/nanopore/UCSC_HPRC_nanopore_Year4/bas...,['/private/groups/hprc/human-pangenomics/1000G...,ont,other,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,/private/groups/hprc/human-pangenomics/HPRC_me...,0.411664,...,0.08,0.02,0.0,1,21.48,17.80,15.95,12.78,2.80,